### Harry Potter Invisibility Cloak

<img src="C:/Users/biby_/Desktop/Projects/harry-potter-invisibility-cloak/harrygif.gif"">

#### Steps
1. Record a video with a stable background.
2. Capture and preprocess frames from the video to create an average background.
3. Implement color filtering to isolate the chosen object color (e.g., red) in each frame.
4. Use the color-filtered mask to segment the object from the frame.
5. Combine the segmented object with the preprocessed background to create the final output frame.

In [3]:
import cv2 
import numpy as np 
import time 

In [ ]:
# Creating a VideoCapture object
cap = cv2.VideoCapture("file-path/video-2.mp4")

# Check if the video capture object is opened successfully
if not cap.isOpened():
    print("Error opening video stream or file")
    exit()

# Try to read a frame to check if it's working
ret, img = cap.read()
if not ret:
    print("Failed to read a frame from the video.")
    cap.release()
    exit()



In [ ]:
# Now you can check the shape of the image
print(img.shape)

In [20]:
# Reset the video capture to the start
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

# We give some time for the camera to warm-up!
time.sleep(3)

background = None

# Read frames to get a suitable background
for i in range(30):
    ret, frame = cap.read()
    if not ret:
        print("Failed to read a frame from the video.")
        cap.release()
        exit()

    background = frame

# Check if background is not None before flipping
if background is not None:
    # Laterally invert the image / flip the image.
    background = np.flip(background, axis=1)

    # Check dimensions after flipping
    print("After flip:", background.shape)
else:
    print("Failed to obtain a suitable background.")

# Get frame width and height
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# Use XVID codec and correct frame resolution
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter("file-path/red-output.avi", fourcc, 20.0, (frame_width, frame_height))

# Reset the video capture to the start
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

while cap.isOpened():
    # Capturing the live frame
    ret, img = cap.read()

    # Check if the frame is read correctly
    if not ret:
        print("Reached the end of the video.")
        break

    # Laterally invert the image / flip the image
    img = np.flip(img, axis=1)

    # converting from BGR to HSV color space
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # Range for lower red
    lower_red = np.array([0, 120, 70])
    upper_red = np.array([10, 255, 255])
    mask1 = cv2.inRange(hsv, lower_red, upper_red)

    # Range for upper range
    lower_red = np.array([170, 120, 70])
    upper_red = np.array([180, 255, 255])
    mask2 = cv2.inRange(hsv, lower_red, upper_red)

    # Generating the final mask to detect red color
    mask = mask1 + mask2

    mask1 = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8))
    mask1 = cv2.morphologyEx(mask1, cv2.MORPH_DILATE, np.ones((3, 3), np.uint8))

    # creating an inverted mask to segment out the cloth from the frame
    mask2 = cv2.bitwise_not(mask1)

    # Segmenting the cloth out of the frame using bitwise and with the inverted mask
    res1 = cv2.bitwise_and(img, img, mask=mask2)

    # creating image showing static background frame pixels only for the masked region
    res2 = cv2.bitwise_and(background, background, mask=mask1)

    # Generating the final output
    final_output = cv2.addWeighted(res1, 1, res2, 1, 0)

    cv2.imshow("magic", final_output)

    # Write the processed frame to the output file
    out.write(final_output)

    k = cv2.waitKey(10)
    if k == 27:  # Press 'ESC' to exit
        break

cap.release()
out.release()
cv2.destroyAllWindows()


(1280, 1280, 3)
After flip: (1280, 1280, 3)
Reached the end of the video.
